# CryptoCTF 2019: roXen

__Tags:__ `rsa`  
__Total Points:__ 159  
__Total Solvers:__ 22 

# Problem Statement

You are given the source code used for encrypting and the corresponding output for `c` and `n`

```python
#!/usr/bin/env python

from Crypto.Util.number import *
from secret import exp, flag, nbit

assert exp & (exp + 1) == 0

def adlit(x):
    l = len(bin(x)[2:])
    return (2 ** l - 1) ^ x

def genadlit(nbit):
    while True:
        p = getPrime(nbit)
        q = adlit(p) + 31337
        if isPrime(q):
            return p, q

p, q = genadlit(nbit)
e, n = exp, p * q

c = pow(bytes_to_long(flag), e, n)

print 'n =', hex(n)
print 'c =', hex(c)
```

# Solution Overview

Because of the way the `p` and `q` are generated, we realize that possible values `p + q` are limited. This makes factorizing `N`  into `p` and `q` feasible.

Since `e` is not known we have to guess it after getting `p` and `q`. Because of the assertion that `exp & (exp + 1) == 0`, the possible values of `e` are limited to the form `2**k - 1`

With limited search space for `p`, `q`, and `e`, we brute force to get the flag!

# Solution

## Factoring N

Whenever there are RSA problems, one of the first things I look at is the generation of the prime numbers `p` and `q`. 

The proper way to generate it is independently

```python
p = getPrime(nbit)
q = getPrime(nbit)
```

If `q` is somehow derived from `p`, then this probably introduces some weakness. One simpler example of this is:

```python
p = getPrime(nbit)
q = next_prime(p)
```

### Analyzing adlit

Since `adlit` is used to derive `q` from `p`, we look at how predictable it is

In [7]:
def adlit(x):
    l = len(bin(x)[2:])
    return (2 ** l - 1) ^ x

The value being _xor-ed_ to `x` is constant with respect to the number of bits of `x`.  `2 ** l - 1` is an idiom for producing a binary number of all ones. So `adlit` just flips all the bits of `x`

In [20]:
print('37       ', bin(37)[2:])
print('adlit(37)', bin(adlit(37))[2:].zfill(6))

37        100101
adlit(37) 011010


This means that the `x + adlit(x)` is constant depending on the binary length of `x`. 

$ x + adlit(x) = 2 ^ k - 1 \tag{4.1.1.1}$ 

Where `k` is the binary length of `x`. 

In [31]:
37 + adlit(37)

63

In [32]:
269 + adlit(269)

511

And from the `genadlit`, we know that

$q = adlit(p) + 31337 \tag{4.1.1.2}$  

With (4.1.1.1) and (4.1.1.2), can easily derive a form for `p + q`

$$
p + q = p + (adlit(p) + 31337)) 
$$

$$
= (p + adlit(p)) + 31337
$$

$$
= (2 ^ k - 1) + 31337
$$

Therefore, for some integer positive integer `k`,

$$
p + q = 2 ^ k + 31336 \tag{4.1.1.3}
$$

### Getting p and q

Now we have two equations for `p` and `q`, using `n` and (4.1.1.3),

$$ p + q = 2 ^ k + 31336 $$

$$ n = p q $$


And if we rename $ R = 2 ^ k + 31336 $, solving for `p` we get the quadtratic equation:

$ p^2 - Rp + n = 0 \tag{4.1.2.1} $ 

In [63]:
import gmpy2 

n = 0x3ff77ad8783e006b6a2c9857f2f13a9d896297558e7c986c491e30c1a920512a0bad9f07c5569cf998fc35a3071de9d8b0f5ada4f8767b828e35044abce5dcf88f80d1c0a0b682605cce776a184e1bcb8118790fff92dc519d24f998a9c04faf43c434bef6c0fa39a3db7452dc07ccfced9271799f37d91d56b5f21c51651d6a9a41ee5a8af17a2f945fac2b1a0ea98bc70ef0f3e37371c9c7b6f90d3d811212fc80e0abcd5bbefe0c6edb3ca6845ded90677ccd8ff4de2c747b37265fc1250ba9aa89b4fd2bdfb4b4b72a7ff5b5ee67e81fd25027b6cb49db610ec60a05016e125ce0848f2c32bff33eed415a6d227262b338b0d1f3803d83977341c0d3638f
c = 0x2672cade2272f3024fd2d1984ea1b8e54809977e7a8c70a07e2560f39e6fcce0e292426e28df51492dec67d000d640f3e5b4c6c447845e70d1432a3c816a33da6a276b0baabd0111279c9f267a90333625425b1d73f1cdc254ded2ad54955914824fc99e65b3dea3e365cfb1dce6e025986b2485b6c13ca0ee73c2433cf0ca0265afe42cbf647b5c721a6e51514220bab8fcb9cff570a6922bceb12e9d61115357afe1705bda3c3f0b647ba37711c560b75841135198cc076d0a52c74f9802760c1f881887cc3e50b7e0ff36f0d9fa1bfc66dff717f032c066b555e315cb07e3df13774eaa70b18ea1bb3ea0fd1227d4bac84be2660552d3885c79815baef661

def get_pq(k):
    R = 2**k + 31336

    temp = R**2 - 4*n
    if temp < 0: return -1, -1
    sq, flag = gmpy2.iroot(temp, 2)
    if not flag: return -1, -1
    p = (R + sq) // 2
    q = (R - sq) // 2
    return p, q

k = len(bin(n)[2:]) // 2 + 1
p, q = get_pq(k)
phi = (p-1)*(q-1) 

print('nbit =', k)
print('p    =', p)
print('q    =', q)


nbit = 1024
p    = 91934396941118575436929554782758166784623142015203107928295225306949429527662253180027648166060067602233902389535868116051536080388999480377007211745229221564969130373120800620379012435790356909945473565305296926519232706950561924532325538399351352696805684504904629096892037592742285758390953849377910498739
q    = 87834916545113015336000964296144306577174555879027549345134855850783246277838709952680829156347468418886211490335525241607253688425417142115840218894244902812798763051744684655923207165455737209507609386779708842318917975391900956941587572141475884466544826179681669143055208345737430546444402480246313669813


And some sanity checking

In [66]:
assert p*q == n
assert gmpy2.is_prime(p)
assert gmpy2.is_prime(q)
assert pow(2, phi, n) == 1

## Getting e

The clue for the possible values of `e` is from the assertion

```python
assert exp & (exp + 1) == 0
```

A quick and dirty way to analyze this is to just enumerate the first valid values of `exp`

In [68]:
for exp in range(1, 1000):
    if exp & (exp + 1) == 0:
        print(exp)

1
3
7
15
31
63
127
255
511


We see see that the exponent follows a similar form to what we've seen previously. For some integer `k`

$ e = 2^k - 1 \tag{4.2.1}$

In [70]:
for k in range(1, 100):
    exp = 2**k - 1
    assert exp & (exp + 1) == 0

Although one problem to note is that valid values for `exp` might not be coprime with `phi`. This may be a problem since we need `e` and `phi` to be coprime to be able decrypt. We will address this in the next section.

In [72]:
for k in range(1, 10):
    exp = 2**k - 1
    if gmpy2.gcd(exp, phi) != 1:
        print(k, exp)

2 3
3 7
4 15
6 63
8 255
9 511


## Getting the flag

### The case where e and phi are not coprime

Given a candidate `e` as well as `phi`, and `n`, we can easily decrypt if `e` and `phi` are coprime.  However, if they have a common factor, then we will have to work around it. 

If $e = g*e\prime$ where $g = gcd(e, \phi(n))$ and, $e\prime$ and $\phi(n)$ are coprime, the "best" we can do is to partially decrypt the message using some $ d\prime \equiv e\prime^{-1} \mod \phi(n) $

$$ c \equiv m ^ e \mod n $$
$$ c \equiv m ^ {ge\prime} \mod n $$
$$ c^{d\prime} \equiv m ^ {g e\prime d\prime}  \mod n $$
$$ c^{d\prime} \equiv m ^ {g}  \mod n $$

Now how do we get $m \mod n$ from $m ^ g \mod n$? If $g$ and $m$ are small enough, and $n$ is big enough, ($ m ^ g < n $), then we can just get the root directly! Here is a quick example:

In [105]:
original_number = 1337
test_cipher = pow(original_number, 7, n)
gmpy2.iroot(test_cipher, 7)

(mpz(1337), True)

Since `n` is around 250 bytes long, and if we hope that the flag just around 30 - 40, then we can do this for exponents as high as 7.

### Putting it all together

In [73]:
from Cryptodome.Util.number import long_to_bytes

In [121]:
k = 1
while True:
    k += 1
    e = 2**k - 1

    gcd =  gmpy2.gcd(e, phi)
    
    # Ignore if gcd is too high
    if gcd > 8: continue
    if e % gcd**2 == 0: continue
    e_prime = e // gcd

    d_prime = gmpy2.invert(e_prime, phi)
    
    # RSA decryption
    p_g = pow(c, d_prime, n)
    
    p, is_valid = gmpy2.iroot(p_g, gcd)
    plaintext = long_to_bytes(p)
    if b'CCTF' in plaintext and is_valid:
        break

In [122]:
print('k =', k)
print('flag:', plaintext)

k = 3729
flag: b'CCTF{it5_3a5y_l1k3_5uNd4y_MOrn1N9}'
